# Урок 3. Логистическая регрессия. Log Loss

**задание 1:**

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [13]:
# код с урока
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [35]:
def calc_logloss1(y, y_pred):
    #отловим нули и единицы
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            y_pred[i] += 1e-10
        elif y_pred[i] == 1:
            y_pred[i] -= 1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

**задание 2**

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.



In [10]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [11]:
def robust_scale(X):
    X_rob = X.copy().astype(np.float64)
    for i in range(X.shape[0]):
        if i == 0:
          continue
        percentiles = np.percentile(X[i], (25.0, 75.0))
        X_rob[i] = (X[i] - np.median(X[i])) / percentiles[1] - percentiles[0]
    return X_rob
X_rob = robust_scale(X)
X_rob  

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00],
       [-1.18181818e+00, -1.18181818e+00, -8.18181818e-01,
        -1.18181818e+00, -4.54545455e-01, -1.54545455e+00,
         2.72727273e-01,  2.09090909e+00, -1.18181818e+00,
        -8.18181818e-01],
       [-6.25205607e+02, -6.25056075e+02, -6.25018692e+02,
        -6.25130841e+02, -6.24495327e+02, -6.24981308e+02,
        -6.24457944e+02, -6.24084112e+02, -6.25242991e+02,
        -6.24831776e+02],
       [-1.25000000e+00, -1.25000000e+00, -7.50000000e-01,
        -1.25000000e+00, -7.50000000e-01, -1.25000000e+00,
        -2.50000000e-01, -2.50000000e-01, -1.25000000e+00,
        -7.50000000e-01]])

In [14]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [61]:
#  код с урока
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss1(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    #if i % (iterations / 10) == 0:
        #print(i, W, err)
    return (i, W, err)

In [75]:
W = eval_model(X_rob, y, iterations=1000, alpha=1e-5)
W

(1000,
 array([ 4.97752615e-01, -1.39608623e-01, -1.26874105e-03,  1.52249130e+00]),
 0.607030536477691)

In [121]:
def gradient_descent_optimal_alpha(X, y, epsilon):
    err_border = float('inf')
    for alpha in np.linspace(1e-7, 1e-2, num = 1000):
        i, W, err = eval_model(X, y, iterations=1000, alpha=alpha)
        if (err < err_border):
            i_border = i
            W_border = W
            err_border = err
            alpha_border = alpha
    print(f"для заданной точности {epsilon}\nоптимальное количество итераций = {i_border}\nоптимальная скорость обучения ={alpha_border}\nпри них получаем W = {W_border} и logloss = {err_border}")
    return W_border

In [122]:
W1 = gradient_descent_optimal_alpha(X_rob, y, 1e-4)

для заданной точности 0.0001
оптимальное количество итераций = 1000
оптимальная скорость обучения =2.011981981981982e-05
при них получаем W = [ 4.97752377e-01 -1.40526364e-01 -1.26827263e-03  1.52279973e+00] и logloss = 0.606937925609538


**задание 3:**

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).



In [80]:
def calc_pred_proba(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W, X)))
    return y_pred_proba

In [81]:
calc_pred_proba(W1,X_rob)

array([0.39013625, 0.39009113, 0.56545293, 0.39011369, 0.55269325,
       0.40229216, 0.70489341, 0.64901979, 0.39014753, 0.56539468])

**задание 4:**

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [90]:
def calc_pred(W, X, border=0.55):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred_proba = y_pred_proba.copy()
    y_pred_proba[y_pred_proba >= border] = 1
    y_pred_proba[y_pred_proba < border] = 0
    return y_pred_proba

In [95]:
Y_pred = calc_pred(W1, X_rob, border=0.55)
Y_pred

array([0., 0., 1., 0., 1., 0., 1., 1., 0., 1.])

**задание 5:**

Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [92]:
def accuracy(y, y_pred):
    res = (y_pred == y).sum()/len(y)
    return res

In [96]:
accuracy(y, Y_pred)

0.8

In [101]:
def error_matrix(y, y_pred):
    TP = y[(y - y_pred) == 0].sum()
    FP = ((y - y_pred) == -1).sum()
    FN = ((y - y_pred) == 1).sum()
    TN = (y[(y - y_pred) == 0]==0).sum()
    
    return np.array([[TP, FP], 
                     [FN, TN]])

In [104]:
EM = error_matrix(y, Y_pred)
EM

array([[4., 1.],
       [1., 4.]])

In [109]:
def precision_recall_F1(y, y_pred):
    TP = EM[0,0]
    FP = EM[0,1]
    FN = EM[1,0]
    TN = EM[1,1]
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, F1

In [110]:
precision_recall_F1(y, Y_pred)

(0.8, 0.8, 0.8000000000000002)

**задание 6:**

Могла ли модель переобучиться? Почему?

Я думаю, что модель всегда может переобучиться. Так как нет никаких факторов, сдерживающих это переобучение. В таких случаях может помочь регуляризация.

**задание 7:** *(опция)* 

Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [124]:
def eval_model_reg1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(15)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss1(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W
W = eval_model_reg1(X_rob, y, iterations=10000, alpha=2.011981981981982e-05)

1000 [-3.12577541e-01  3.38572409e-01 -1.07225151e-04 -5.00244879e-01] 0.735757159148055
2000 [-3.12577266e-01  3.37690365e-01 -1.08253946e-04 -4.98928895e-01] 0.7356324155257699
3000 [-3.12576990e-01  3.36810483e-01 -1.09285657e-04 -4.97612447e-01] 0.7355078005639856
4000 [-3.12576713e-01  3.35932754e-01 -1.10320269e-04 -4.96295538e-01] 0.7353833133415323
5000 [-3.12576436e-01  3.35057167e-01 -1.11357764e-04 -4.94978172e-01] 0.7352589529462417
6000 [-3.12576157e-01  3.34183711e-01 -1.12398126e-04 -4.93660353e-01] 0.7351347184748633
7000 [-3.12575877e-01  3.33312377e-01 -1.13441336e-04 -4.92342084e-01] 0.7350106090329855
8000 [-3.12575597e-01  3.32443153e-01 -1.14487379e-04 -4.91023369e-01] 0.734886623734955
9000 [-3.12575315e-01  3.31576029e-01 -1.15536238e-04 -4.89704212e-01] 0.7347627617037963
10000 [-3.12575033e-01  3.30710996e-01 -1.16587896e-04 -4.88384616e-01] 0.7346390220711347


In [123]:
def eval_model_reg2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(15)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss1(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W
W = eval_model_reg2(X_rob, y, iterations=10000, alpha=2.011981981981982e-05)

1000 [-3.12577541e-01  3.38572409e-01 -1.07225152e-04 -5.00244879e-01] 0.7357571591604324
2000 [-3.12577266e-01  3.37690365e-01 -1.08253946e-04 -4.98928895e-01] 0.7356324155505264
3000 [-3.12576991e-01  3.36810483e-01 -1.09285658e-04 -4.97612447e-01] 0.7355078006011105
4000 [-3.12576714e-01  3.35932755e-01 -1.10320270e-04 -4.96295538e-01] 0.7353833133910156
5000 [-3.12576436e-01  3.35057168e-01 -1.11357765e-04 -4.94978172e-01] 0.7352589530080731
6000 [-3.12576158e-01  3.34183712e-01 -1.12398127e-04 -4.93660353e-01] 0.7351347185490333
7000 [-3.12575878e-01  3.33312377e-01 -1.13441338e-04 -4.92342085e-01] 0.7350106091194847
8000 [-3.12575598e-01  3.32443154e-01 -1.14487381e-04 -4.91023370e-01] 0.7348866238337741
9000 [-3.12575316e-01  3.31576030e-01 -1.15536240e-04 -4.89704213e-01] 0.7347627618149267
10000 [-3.12575034e-01  3.30710997e-01 -1.16587898e-04 -4.88384617e-01] 0.7346390221945678
